Pruebas en el modelo v6

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import _init_paths

import os

import torch
import torch.utils.data
from opts import opts
from models.model import create_model, load_model, save_model
from models.data_parallel import DataParallel
from logger import Logger
from datasets.dataset_factory import get_dataset
from trains.train_factory import train_factory
import cv2
import sys
%matplotlib inline
import matplotlib.pyplot as plt
from detectors.detector_factory import detector_factory

image_ext = ['jpg', 'jpeg', 'png', 'webp']
video_ext = ['mp4', 'mov', 'avi', 'mkv']
time_stats = ['tot', 'load', 'pre', 'net', 'dec', 'post', 'merge']

In [20]:
sys.argv = ['-f', 'ctdetv2', '--demo', '../images/yolo/', '--arch','dlav6_34',  '--load_model' ,'../exp/ctdetv2/coco_dlav6/model_best.pth'] #cmd = 'ctdet --exp_id coco_dla --batch_size 4 --lr 1.25e-4  --gpus 0 --num_workers 0'
opt = opts().parse()
print(opt.task)

Fix size testing.
training chunk_sizes: [32]
The output will be saved to  C:\Users\avata\FewShot-CenterNet\src\lib\..\..\exp\ctdetv2\default
ctdetv2


In [21]:
opt = opts().init()

Fix size testing.
training chunk_sizes: [32]
The output will be saved to  C:\Users\avata\FewShot-CenterNet\src\lib\..\..\exp\ctdetv2\default
heads {'hm': 80, 'wh': 2, 'reg': 2}


In [22]:
os.environ['CUDA_VISIBLE_DEVICES'] = opt.gpus_str
#opt.debug = max(opt.debug, 1)
opt.debug = 0
Detector = detector_factory[opt.task]
detector = Detector(opt)

Creating model...
loaded ../exp/ctdetv2/coco_dlav6/model_best.pth, epoch 220


In [23]:
detector.model

DLASeg(
  (base): DLA(
    (base_layer): Sequential(
      (0): Conv2d(3, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (level0): Sequential(
      (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (level1): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (level2): Tree(
      (tree1): BasicBlock(
        (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (re

In [24]:
with torch.no_grad():  
    params = detector.model.named_parameters()
    for nombre_parametro, parametro in params:
        if "fc_ss.3.W" in nombre_parametro:
            W = parametro.data
        if "fc_ss.3.temp" in nombre_parametro:
            temp = parametro

In [25]:
W

tensor([[ 0.0699,  0.1077, -0.2450,  ...,  0.2182, -0.0612, -0.3657],
        [ 0.0627,  0.1483,  0.1455,  ...,  0.1987, -0.3940, -0.2464],
        [ 0.1115,  0.5154,  0.2488,  ..., -0.0270,  0.0724, -0.3536],
        ...,
        [-0.0276,  0.2400, -0.1303,  ...,  0.1793,  0.0223,  0.2665],
        [ 0.1154,  0.1622,  0.0403,  ..., -0.1505,  0.1649, -0.3762],
        [-0.0689,  0.1931, -0.0773,  ..., -0.4931, -0.0268, -0.1302]],
       device='cuda:0')

In [26]:
temp

Parameter containing:
tensor(10.6730, device='cuda:0', requires_grad=True)

In [27]:
with torch.no_grad():  
    params = detector.model.named_parameters()
    for nombre_parametro, parametro in params:
        print(nombre_parametro)

base.base_layer.0.weight
base.base_layer.1.weight
base.base_layer.1.bias
base.level0.0.weight
base.level0.1.weight
base.level0.1.bias
base.level1.0.weight
base.level1.1.weight
base.level1.1.bias
base.level2.tree1.conv1.weight
base.level2.tree1.bn1.weight
base.level2.tree1.bn1.bias
base.level2.tree1.conv2.weight
base.level2.tree1.bn2.weight
base.level2.tree1.bn2.bias
base.level2.tree2.conv1.weight
base.level2.tree2.bn1.weight
base.level2.tree2.bn1.bias
base.level2.tree2.conv2.weight
base.level2.tree2.bn2.weight
base.level2.tree2.bn2.bias
base.level2.root.conv.weight
base.level2.root.bn.weight
base.level2.root.bn.bias
base.level2.project.0.weight
base.level2.project.1.weight
base.level2.project.1.bias
base.level3.tree1.tree1.conv1.weight
base.level3.tree1.tree1.bn1.weight
base.level3.tree1.tree1.bn1.bias
base.level3.tree1.tree1.conv2.weight
base.level3.tree1.tree1.bn2.weight
base.level3.tree1.tree1.bn2.bias
base.level3.tree1.tree2.conv1.weight
base.level3.tree1.tree2.bn1.weight
base.leve

In [3]:
test = torch.ones(256,80)

In [6]:
x = torch.rand(128,256)

In [28]:
W.shape 

torch.Size([256, 80])

In [29]:
class_name_coco = [
      '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
      'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant',
      'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
      'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack',
      'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
      'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
      'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass',
      'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
      'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
      'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv',
      'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
      'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
      'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [30]:
class_name_caltech80 = [
      'ak47', 'american-flag', 'backpack', 'baseball-bat', 'baseball-glove', 'bear',
      'beer-mug', 'binoculars', 'blimp', 'bonsai', 'boom-box', 'bowling-ball',
      'bowling-pin', 'boxing-glove', 'brain', 'breadmaker', 'buddha', 'butterfly',
      'cactus', 'cake', 'calculator', 'camel', 'canoe', 'car-tire', 'cartman',
      'cereal-box', 'chess-board', 'cockroach', 'coffin', 'coin', 'computer-keyboard',
      'computer-monitor', 'computer-mouse', 'cowboy-hat', 'crab', 'desk-globe',
      'dice', 'dolphin', 'electric-guitar', 'elk', 'faces-easy', 'fire-extinguisher',
      'floppy-disk', 'frying-pan', 'gas-pump', 'golf-ball', 'grapes', 'hamburger',
      'homer-simpson', 'hot-air-ballon', 'hot-dog', 'hourglass', 'ice-cream-cone',
      'iris', 'mars', 'megaphone', 'microwave', 'motorbike', 'owl', 'photocopier',
      'picnic-table', 'playing-card', 'refrigerator', 'revolver', 'saturn', 'snail',
      'soccer-ball', 'soda-can', 'spaghetti', 'spider', 'starfish', 'steering-wheel',
      'swan', 'swiss-army-knife', 'tennis-ball', 'tennis-racket', 'toaster', 'top-hat',
      'vcr','washing-machine']

In [32]:
for i, name in enumerate(class_name_coco):
    if name == 'stop sign':
        print(i)
        print(class_name_caltech80[i-1])
        print(class_name_caltech80[i])
        print(class_name_caltech80[i+1])

12
bowling-ball
bowling-pin
boxing-glove


In [31]:
opt.demo = '../images/256_ObjectCategories/Caltech80/train/'
image_names = [opt.demo]
#image_names.append('../images/Perro1.jpg')
"""
image_names.append('../images/aceite_4.png')
image_names.append('../images/cubix_0.png')
image_names.append('../images/cubix_5.png')
image_names.append('../images/te_0.png')
image_names.append('../images/te_8.png')
"""
images_path = {}
ls = os.listdir(opt.demo)

for folder_name in sorted(ls):
    image_names = []
    ls2 = os.listdir(os.path.join(opt.demo, folder_name))
    for file_name in sorted(ls2):
      ext = file_name[file_name.rfind('.') + 1:].lower()
      if ext in image_ext:
          image_names.append(os.path.join(opt.demo, folder_name, file_name).replace("\\","/"))
    images_path[folder_name] = image_names
"""
images_path['ak47'] :
 ['../images/256_ObjectCategories/Caltech80/train/ak47/001_0002.jpg',
  '../images/256_ObjectCategories/Caltech80/train/ak47/001_0003.jpg',
  '../images/256_ObjectCategories/Caltech80/train/ak47/001_0029.jpg',
  '../images/256_ObjectCategories/Caltech80/train/ak47/001_0036.jpg',
  '../images/256_ObjectCategories/Caltech80/train/ak47/001_0058.jpg'],
"""

"\nimages_path['ak47'] :\n ['../images/256_ObjectCategories/Caltech80/train/ak47/001_0002.jpg',\n  '../images/256_ObjectCategories/Caltech80/train/ak47/001_0003.jpg',\n  '../images/256_ObjectCategories/Caltech80/train/ak47/001_0029.jpg',\n  '../images/256_ObjectCategories/Caltech80/train/ak47/001_0036.jpg',\n  '../images/256_ObjectCategories/Caltech80/train/ak47/001_0058.jpg'],\n"

In [32]:
for index_w, folder in enumerate(images_path.keys()): 
    print(folder)
    for (image_name) in images_path[folder]:
        print(image_name)

ak47
../images/256_ObjectCategories/Caltech80/train/ak47/001_0002.jpg
../images/256_ObjectCategories/Caltech80/train/ak47/001_0003.jpg
../images/256_ObjectCategories/Caltech80/train/ak47/001_0009.jpg
../images/256_ObjectCategories/Caltech80/train/ak47/001_0021.jpg
../images/256_ObjectCategories/Caltech80/train/ak47/001_0022.jpg
../images/256_ObjectCategories/Caltech80/train/ak47/001_0029.jpg
../images/256_ObjectCategories/Caltech80/train/ak47/001_0036.jpg
../images/256_ObjectCategories/Caltech80/train/ak47/001_0039.jpg
../images/256_ObjectCategories/Caltech80/train/ak47/001_0058.jpg
../images/256_ObjectCategories/Caltech80/train/ak47/001_0071.jpg
american-flag
../images/256_ObjectCategories/Caltech80/train/american-flag/002_0001.jpg
../images/256_ObjectCategories/Caltech80/train/american-flag/002_0007.jpg
../images/256_ObjectCategories/Caltech80/train/american-flag/002_0013.jpg
../images/256_ObjectCategories/Caltech80/train/american-flag/002_0020.jpg
../images/256_ObjectCategories/Calt

In [33]:
W_b = torch.zeros((256,80))
m = torch.nn.ReLU()
for index_w, folder in enumerate(images_path.keys()): 
    for (image_name) in images_path[folder]:
        ret = detector.run(image_name)
        output = ret['output']
        pred_obj = output['objns']
        hm = pred_obj.squeeze(0).squeeze(0)
        scores = output['hm'].squeeze(0)
        batch, cat, height, width = pred_obj.size()
        topk_scores, topk_inds = torch.topk(pred_obj.view(batch, cat, -1), 1)
        print(topk_scores)
        print(topk_inds)

        topk_inds = topk_inds % (height * width)
        topk_ys   = (topk_inds / width).int()
        topk_xs   = (topk_inds % width).int()
        
        
        print(topk_ys[0,0,0])
        print(topk_xs[0,0,0])
        print(hm[topk_ys[0,0,0],topk_xs[0,0,0]])
        print(max(scores[:,topk_ys[0,0,0],topk_xs[0,0,0]]))
        #descriptores[image_name] = output['descriptor'].squeeze(0)[:,int(topk_xs),int(topk_ys)]
        radio = 2
        eje_x, eje_y = int(topk_xs), int(topk_ys)
        #eje_x, eje_y = 62,64

        descriptores = output['descriptor'].squeeze(0)[:,eje_x,eje_y]
        count = 1
        for x in range(eje_x - radio, eje_x + radio+1):
            for y in range(eje_y - radio, eje_y + radio+1):
                if not (x==eje_x and y==eje_y):
                    descriptores = (descriptores+output['descriptor'].squeeze(0)[:,x,y])
                    count += 1
        W_b[:,index_w] = descriptores/count
        print('-------------------------')

tensor([[[0.9895]]], device='cuda:0')
tensor([[[3398]]], device='cuda:0')
tensor(26, device='cuda:0', dtype=torch.int32)
tensor(70, device='cuda:0', dtype=torch.int32)
tensor(0.9895, device='cuda:0')
tensor(0.2783, device='cuda:0')
-------------------------
tensor([[[0.9999]]], device='cuda:0')
tensor([[[8255]]], device='cuda:0')
tensor(64, device='cuda:0', dtype=torch.int32)
tensor(63, device='cuda:0', dtype=torch.int32)
tensor(0.9999, device='cuda:0')
tensor(0.4031, device='cuda:0')
-------------------------
tensor([[[0.9999]]], device='cuda:0')
tensor([[[8408]]], device='cuda:0')
tensor(65, device='cuda:0', dtype=torch.int32)
tensor(88, device='cuda:0', dtype=torch.int32)
tensor(0.9999, device='cuda:0')
tensor(0.1493, device='cuda:0')
-------------------------
tensor([[[0.9998]]], device='cuda:0')
tensor([[[8776]]], device='cuda:0')
tensor(68, device='cuda:0', dtype=torch.int32)
tensor(72, device='cuda:0', dtype=torch.int32)
tensor(0.9998, device='cuda:0')
tensor(0.1344, device='cud

tensor([[[8127]]], device='cuda:0')
tensor(63, device='cuda:0', dtype=torch.int32)
tensor(63, device='cuda:0', dtype=torch.int32)
tensor(0.9999, device='cuda:0')
tensor(0.1986, device='cuda:0')
-------------------------
tensor([[[0.9998]]], device='cuda:0')
tensor([[[8127]]], device='cuda:0')
tensor(63, device='cuda:0', dtype=torch.int32)
tensor(63, device='cuda:0', dtype=torch.int32)
tensor(0.9998, device='cuda:0')
tensor(0.2478, device='cuda:0')
-------------------------
tensor([[[0.9999]]], device='cuda:0')
tensor([[[8127]]], device='cuda:0')
tensor(63, device='cuda:0', dtype=torch.int32)
tensor(63, device='cuda:0', dtype=torch.int32)
tensor(0.9999, device='cuda:0')
tensor(0.1745, device='cuda:0')
-------------------------
tensor([[[0.9999]]], device='cuda:0')
tensor([[[8127]]], device='cuda:0')
tensor(63, device='cuda:0', dtype=torch.int32)
tensor(63, device='cuda:0', dtype=torch.int32)
tensor(0.9999, device='cuda:0')
tensor(0.1187, device='cuda:0')
-------------------------
tensor

tensor([[[0.9842]]], device='cuda:0')
tensor([[[7148]]], device='cuda:0')
tensor(55, device='cuda:0', dtype=torch.int32)
tensor(108, device='cuda:0', dtype=torch.int32)
tensor(0.9842, device='cuda:0')
tensor(0.1283, device='cuda:0')
-------------------------
tensor([[[0.9639]]], device='cuda:0')
tensor([[[8647]]], device='cuda:0')
tensor(67, device='cuda:0', dtype=torch.int32)
tensor(71, device='cuda:0', dtype=torch.int32)
tensor(0.9639, device='cuda:0')
tensor(0.1569, device='cuda:0')
-------------------------
tensor([[[0.9999]]], device='cuda:0')
tensor([[[8384]]], device='cuda:0')
tensor(65, device='cuda:0', dtype=torch.int32)
tensor(64, device='cuda:0', dtype=torch.int32)
tensor(0.9999, device='cuda:0')
tensor(0.1298, device='cuda:0')
-------------------------
tensor([[[0.9986]]], device='cuda:0')
tensor([[[8255]]], device='cuda:0')
tensor(64, device='cuda:0', dtype=torch.int32)
tensor(63, device='cuda:0', dtype=torch.int32)
tensor(0.9986, device='cuda:0')
tensor(0.3487, device='cu

tensor([[[14283]]], device='cuda:0')
tensor(111, device='cuda:0', dtype=torch.int32)
tensor(75, device='cuda:0', dtype=torch.int32)
tensor(0.9982, device='cuda:0')
tensor(0.0889, device='cuda:0')
-------------------------
tensor([[[0.9999]]], device='cuda:0')
tensor([[[8255]]], device='cuda:0')
tensor(64, device='cuda:0', dtype=torch.int32)
tensor(63, device='cuda:0', dtype=torch.int32)
tensor(0.9999, device='cuda:0')
tensor(0.5459, device='cuda:0')
-------------------------
tensor([[[0.9576]]], device='cuda:0')
tensor([[[8315]]], device='cuda:0')
tensor(64, device='cuda:0', dtype=torch.int32)
tensor(123, device='cuda:0', dtype=torch.int32)
tensor(0.9576, device='cuda:0')
tensor(0.3364, device='cuda:0')
-------------------------
tensor([[[0.9987]]], device='cuda:0')
tensor([[[8136]]], device='cuda:0')
tensor(63, device='cuda:0', dtype=torch.int32)
tensor(72, device='cuda:0', dtype=torch.int32)
tensor(0.9987, device='cuda:0')
tensor(0.3295, device='cuda:0')
-------------------------
ten

tensor([[[8127]]], device='cuda:0')
tensor(63, device='cuda:0', dtype=torch.int32)
tensor(63, device='cuda:0', dtype=torch.int32)
tensor(0.9965, device='cuda:0')
tensor(0.1009, device='cuda:0')
-------------------------
tensor([[[0.9985]]], device='cuda:0')
tensor([[[8254]]], device='cuda:0')
tensor(64, device='cuda:0', dtype=torch.int32)
tensor(62, device='cuda:0', dtype=torch.int32)
tensor(0.9985, device='cuda:0')
tensor(0.2039, device='cuda:0')
-------------------------
tensor([[[0.9999]]], device='cuda:0')
tensor([[[8127]]], device='cuda:0')
tensor(63, device='cuda:0', dtype=torch.int32)
tensor(63, device='cuda:0', dtype=torch.int32)
tensor(0.9999, device='cuda:0')
tensor(0.1034, device='cuda:0')
-------------------------
tensor([[[0.9999]]], device='cuda:0')
tensor([[[8255]]], device='cuda:0')
tensor(64, device='cuda:0', dtype=torch.int32)
tensor(63, device='cuda:0', dtype=torch.int32)
tensor(0.9999, device='cuda:0')
tensor(0.5740, device='cuda:0')
-------------------------
tensor

tensor([[[0.9998]]], device='cuda:0')
tensor([[[12863]]], device='cuda:0')
tensor(100, device='cuda:0', dtype=torch.int32)
tensor(63, device='cuda:0', dtype=torch.int32)
tensor(0.9998, device='cuda:0')
tensor(0.0780, device='cuda:0')
-------------------------
tensor([[[0.9999]]], device='cuda:0')
tensor([[[13120]]], device='cuda:0')
tensor(102, device='cuda:0', dtype=torch.int32)
tensor(64, device='cuda:0', dtype=torch.int32)
tensor(0.9999, device='cuda:0')
tensor(0.0974, device='cuda:0')
-------------------------
tensor([[[0.9684]]], device='cuda:0')
tensor([[[7999]]], device='cuda:0')
tensor(62, device='cuda:0', dtype=torch.int32)
tensor(63, device='cuda:0', dtype=torch.int32)
tensor(0.9684, device='cuda:0')
tensor(0.4426, device='cuda:0')
-------------------------
tensor([[[0.9762]]], device='cuda:0')
tensor([[[9023]]], device='cuda:0')
tensor(70, device='cuda:0', dtype=torch.int32)
tensor(63, device='cuda:0', dtype=torch.int32)
tensor(0.9762, device='cuda:0')
tensor(0.1917, device=

IndexError: index 128 is out of bounds for dimension 1 with size 128

In [35]:
W_b

tensor([[ 0.3900,  3.9005,  2.6874,  ...,  0.0000,  0.0000,  0.0000],
        [-1.1930, -8.9616,  1.7380,  ...,  0.0000,  0.0000,  0.0000],
        [-3.9587, -0.0951,  0.8599,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-2.4652,  2.5262, -2.2561,  ...,  0.0000,  0.0000,  0.0000],
        [-2.0686, -0.4286,  4.1869,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0619,  0.8782, -6.3605,  ...,  0.0000,  0.0000,  0.0000]])

In [36]:
W_b = W_b/temp
print(W_b)

tensor([[ 0.0365,  0.3655,  0.2518,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1118, -0.8397,  0.1628,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3709, -0.0089,  0.0806,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.2310,  0.2367, -0.2114,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1938, -0.0402,  0.3923,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0058,  0.0823, -0.5959,  ...,  0.0000,  0.0000,  0.0000]],
       grad_fn=<DivBackward0>)


In [37]:
with torch.no_grad():  
    params = detector.model.named_parameters()
    for nombre_parametro, parametro in params:
        if "fc_ss.3.W" in nombre_parametro:
            parametro.data = W_b

In [38]:
with torch.no_grad():  
    params = detector.model.named_parameters()
    for nombre_parametro, parametro in params:
        if "fc_ss.3.W" in nombre_parametro:
            print(parametro.data)

tensor([[ 0.0365,  0.3655,  0.2518,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1118, -0.8397,  0.1628,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3709, -0.0089,  0.0806,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.2310,  0.2367, -0.2114,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1938, -0.0402,  0.3923,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0058,  0.0823, -0.5959,  ...,  0.0000,  0.0000,  0.0000]])


In [39]:
torch.save(detector.model.state_dict(), "model_wb_10_n.pth")